In [21]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from itertools import product
import numpy as np
import pandas as pd
import warnings
import matplotlib as plt
import seaborn as sns

warnings.filterwarnings("ignore")


## Import Data

In [15]:
serie = pd.read_csv('data/serie_Demanda_Electrica_UE.csv', index_col=0, parse_dates=True)
serie = serie.asfreq('D')
serie.index

DatetimeIndex(['1979-01-01', '1979-01-02', '1979-01-03', '1979-01-04',
               '1979-01-05', '1979-01-06', '1979-01-07', '1979-01-08',
               '1979-01-09', '1979-01-10',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', name='Date', length=16802, freq='D')

## Modelos ARIMA(p,0,q)

In [ ]:
def obtener_df_modelos_ARIMA(serie_temp, p_values:list, q_values:list, d:int):
    
    # Train-test split
    train_size = int(len(serie_temp) * 0.8)
    train, test = serie_temp[:train_size], serie_temp[train_size:]

    # Crear combinaciones de parámetros
    param_combinations = list(product(p_values, q_values))
    P, D, Q, m = 0, 0, 0, 0  # no estacionalidad 

    # Resultados
    results = []

    # Loop por combinaciones
    for i, (p, q) in enumerate(param_combinations):
        try:
            print()
            print(f'Entrenando modelo ARIMA({p},{d},{q})({P},{D},{Q})[{m}]...')
            model = SARIMAX(train,
                            order=(p, d, q),
                            seasonal_order=(P, D, Q, m),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            fitted_model = model.fit(disp=False)  

            # Forecast
            print('Prediciendo...')
            pred = fitted_model.forecast(steps=len(test))

            # Evaluación
            print('Evaluando...')
            aic = fitted_model.aic
            bic = fitted_model.bic
            rmse = np.sqrt(mean_squared_error(test, pred))
            mape = mean_absolute_percentage_error(test, pred)
            r2 = r2_score(test, pred)

            results.append({
                'p': p, 'q': q, #'P': P, 'Q': Q,
                'AIC': aic,
                'BIC': bic,
                'RMSE': rmse,
                'MAPE': mape,
                'R2': r2
            })

            print(f'Modelo {i+1}/{len(param_combinations)} listo.')

        except Exception as e:
            print(f'Error en modelo ARIMA({p},{d},{q})({P},{D},{Q})[{m}]: {e} de tipo {type(e).__name__}')
            continue

    # Convertir a DataFrame y ordenar por AIC
    df_arima = pd.DataFrame(results).sort_values('AIC')
    return df_arima

In [22]:
# MODELO CON PARÁMETROS TEÓRICAMENTE SIGNIFICATIVOS
serie_ej_01 = serie['2019':'2024']

df_arima_01 = obtener_df_modelos_ARIMA(serie_ej_01, p_values=[1,6,8], q_values=[7,14], d=0)
df_arima_01


Entrenando modelo SARIMA(1,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 1/6 listo: SARIMA(1,0,7)(0,0,0)[0]

Entrenando modelo SARIMA(1,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 2/6 listo: SARIMA(1,0,14)(0,0,0)[0]

Entrenando modelo SARIMA(6,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 3/6 listo: SARIMA(6,0,7)(0,0,0)[0]

Entrenando modelo SARIMA(6,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 4/6 listo: SARIMA(6,0,14)(0,0,0)[0]

Entrenando modelo SARIMA(8,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 5/6 listo: SARIMA(8,0,7)(0,0,0)[0]

Entrenando modelo SARIMA(8,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 6/6 listo: SARIMA(8,0,14)(0,0,0)[0]


,p,q,AIC,BIC,RMSE,MAPE,R2
5,8,14,47797.895053,47923.486330,9.670483e+05,0.079339,9.373331e-03
3,6,14,48847.233447,48961.903743,8.529135e+05,0.073655,2.294094e-01
2,6,7,49228.131881,49304.635019,1.016670e+06,0.087622,-9.489820e-02
4,8,7,49378.058679,49465.490836,8.104841e+10,3092.046701,-6.958293e+09
1,1,14,50395.030610,50482.398455,1.088818e+06,0.095577,-2.558115e-01
0,1,7,50794.772667,50843.953256,1.568578e+06,0.140723,-1.606310e+00


In [ ]:
# Metemos algún parámetro chorra para ver que efectivamente los tóricos están por encima

df_arima_02 = obtener_df_modelos_ARIMA(serie_ej_01, p_values=[1,3,6,7,8], q_values=[2,7,14], d=0)


Entrenando modelo ARIMA(1,0,2)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 1/15 listo: ARIMA(1,0,2)(0,0,0)[0]

Entrenando modelo ARIMA(1,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 2/15 listo: ARIMA(1,0,7)(0,0,0)[0]

Entrenando modelo ARIMA(1,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 3/15 listo: ARIMA(1,0,14)(0,0,0)[0]

Entrenando modelo ARIMA(3,0,2)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 4/15 listo: ARIMA(3,0,2)(0,0,0)[0]

Entrenando modelo ARIMA(3,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 5/15 listo: ARIMA(3,0,7)(0,0,0)[0]

Entrenando modelo ARIMA(3,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 6/15 listo: ARIMA(3,0,14)(0,0,0)[0]

Entrenando modelo ARIMA(6,0,2)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 7/15 listo: ARIMA(6,0,2)(0,0,0)[0]

Entrenando modelo ARIMA(6,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 8/15 listo: ARIMA(6,0,7)(0,0,0)[0]

Entrenando modelo ARIMA(6,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Mo

,p,q,AIC,BIC,RMSE,MAPE,R2
14,8,14,47797.895053,47923.486330,9.670483e+05,0.079339,9.373331e-03
11,7,14,47807.242476,47927.373263,8.694912e+05,0.071824,1.991631e-01
10,7,7,48199.245468,48281.213116,8.757534e+05,0.074445,1.875862e-01
12,8,2,48345.201820,48405.311428,1.124545e+06,0.098349,-3.395765e-01
8,6,14,48847.233447,48961.903743,8.529135e+05,0.073655,2.294094e-01
9,7,2,48919.861996,48974.512823,1.027855e+06,0.089207,-1.191208e-01
7,6,7,49228.131881,49304.635019,1.016670e+06,0.087622,-9.489820e-02
13,8,7,49378.058679,49465.490836,8.104841e+10,3092.046701,-6.958293e+09
6,6,2,49690.003429,49739.194327,1.018189e+06,0.088462,-9.817125e-02
5,3,14,50146.353981,50244.642807,1.059923e+06,0.092760,-1.900428e-01


In [25]:
df_arima_02

,p,q,AIC,BIC,RMSE,MAPE,R2
14,8,14,47797.895053,47923.486330,9.670483e+05,0.079339,9.373331e-03
11,7,14,47807.242476,47927.373263,8.694912e+05,0.071824,1.991631e-01
10,7,7,48199.245468,48281.213116,8.757534e+05,0.074445,1.875862e-01
12,8,2,48345.201820,48405.311428,1.124545e+06,0.098349,-3.395765e-01
8,6,14,48847.233447,48961.903743,8.529135e+05,0.073655,2.294094e-01
9,7,2,48919.861996,48974.512823,1.027855e+06,0.089207,-1.191208e-01
7,6,7,49228.131881,49304.635019,1.016670e+06,0.087622,-9.489820e-02
13,8,7,49378.058679,49465.490836,8.104841e+10,3092.046701,-6.958293e+09
6,6,2,49690.003429,49739.194327,1.018189e+06,0.088462,-9.817125e-02
5,3,14,50146.353981,50244.642807,1.059923e+06,0.092760,-1.900428e-01


In [26]:
# DATOS COMPLETOS
df_arima_fin = obtener_df_modelos_ARIMA(serie, p_values=[1,6,8], q_values=[7,14], d=0)
df_arima_fin


Entrenando modelo ARIMA(1,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 1/6 listo: ARIMA(1,0,7)(0,0,0)[0]

Entrenando modelo ARIMA(1,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 2/6 listo: ARIMA(1,0,14)(0,0,0)[0]

Entrenando modelo ARIMA(6,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 3/6 listo: ARIMA(6,0,7)(0,0,0)[0]

Entrenando modelo ARIMA(6,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 4/6 listo: ARIMA(6,0,14)(0,0,0)[0]

Entrenando modelo ARIMA(8,0,7)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 5/6 listo: ARIMA(8,0,7)(0,0,0)[0]

Entrenando modelo ARIMA(8,0,14)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 6/6 listo: ARIMA(8,0,14)(0,0,0)[0]


,p,q,AIC,BIC,RMSE,MAPE,R2
5,8,14,369292.250822,369464.864635,1.813057e+06,1.613428e-01,-2.255272e+00
4,8,7,374194.401341,374314.488855,1.673384e+32,2.949723e+24,-2.773038e+52
3,6,14,377775.137070,377932.740986,2.560153e+06,2.377233e-01,-5.490773e+00
2,6,7,378987.420107,379092.496682,2.883249e+06,2.718804e-01,-7.232446e+00
1,1,14,389224.074485,389344.153660,2.381492e+06,2.208685e-01,-4.616462e+00
0,1,7,391166.323578,391233.872805,5.158815e+06,5.190680e-01,-2.535509e+01
